In [116]:
import numpy as np
import pandas as pd
from functions import preprocess, activations, losses

In [117]:
df = pd.read_csv('datasets/Train.csv')
df = df.dropna(axis=1).drop(['ID','country','population'],axis=1)
df2 = preprocess.min_max(df)

In [118]:
features, target = df2.loc[:,'Q1':'Q25'], df2['target']

X, Y = features.values.T, target.values
x_train, y_train, x_test, y_test = preprocess.split(X, Y)

# Multi-layered Model
<img src='images/net.png' width='500px'>

# Modelling

In [119]:
def initiate_model(X, Y, learning_rate=0.001, hidden_layers=[10,10]):
    
    n_features = X.shape[0]
    layers = [n_features, *hidden_layers, 1]
    n_layers = len(layers)
    
    model = {
        'learning_rate':learning_rate, 
        'layers':layers, 
        'Y':Y, 
        'loss':0,
         'W':{}, 'B':{}, 'Z':{}, 'A':{0:X},
        'dW':{},'dB':{},'dZ':{},'dA':{},
        }

    for layer in range(1, n_layers):
        
        model['W'][layer] = np.random.randn(layers[layer], layers[layer-1]) / np.sqrt(layers[layer-1])
        model['B'][layer] = np.zeros((layers[layer], 1))
                       
    return model

In [120]:
def forward_1_layer(W, A_min, B, act=activations.sigmoid):
    
    Z = W.dot(A_min) + B 
    A = act(Z)
    
    return Z, A


def forwarding(model):

    for i in range(1, n_layers):
        
        model['Z'][i], model['A'][i] = forward_1_layer(
            W =     model['W'][i], 
            A_min = model['A'][i-1], 
            B =     model['B'][i], 
            act =   activations.sigmoid)
     
    model['loss'] = losses.binary_crossentropy(model['A'][final_layer_index], model['Y'])
        
    return model

In [121]:
def backward_1_layer(W, A, dA, B, A_min, Z, learning_rate, actd=activations.dRelu):
    
    dZ = dA * Z
    dA_min = np.dot(W.T, dZ)
    dW = 1 / A_min.shape[1] * np.dot(dZ, A_min.T)
    dB = 1 / A_min.shape[1] * np.dot(dZ, np.ones([dZ.shape[1], 1]))
    
    W = W - learning_rate * dW
    B = B - learning_rate * dB

    return dA_min, W, B


def backwarding(model):

    y_real = model['Y']
    y_pred = model['A'][final_layer_index]
    extra = 10e-5 #Against zero division
    
    model['dA'][final_layer_index] = (-y_real / (y_pred + extra)) - ((1-y_real)/(1-y_pred + extra))

    for i in range(n_layers -1, 0, -1):

        model['dA'][i-1], model['W'][i], model['B'][i] = backward_1_layer(
            W = model['W'][i],
            A = model['A'][i],
            dA = model['dA'][i],
            B = model['B'][i],
            A_min = model['A'][i-1],
            Z = model['Z'][i]
            )
        
    return model

In [127]:
model = initiate_model(x_train, y_train)

n_layers = len(model['layers'])
final_layer_index = n_layers - 1

for i in range(100):
    
    model = forwarding(model)
    
    if (i%10==0):
        print(model['loss']) 
        
    model = backwarding(model)
    

10.902031476184675
10.855715105393058
10.805409866230839
10.750609079283542
10.690685683080012
10.624842167182992
10.552028971633623
10.470802873759936
10.37906032106806
10.273476244600486
